In [1]:
!apt-get install openjdk-11-jdk-headless -qq
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("DataFrame").getOrCreate()
# sc = spark.sparkContext


In [14]:
student_list_1 = [(1, "Manjot", 88, 60), (2, "Saurav", 92, 73), (3, "Sarv", 90, 81)]
print(student_list_1)

# from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# student_schema = StructType([
#                              StructField("Roll_Number", IntegerType(), True),
#                              StructField("Name", StringType(), True),
#                              StructField("Subject1_Marks", IntegerType(), True),
#                              StructField("Subject2_Marks", IntegerType(), True)

# ])
student_schema = ["Roll_Number", "Name", "Maths", "Physics"]
student_df_11 = spark.createDataFrame(student_list_1, schema=student_schema)
student_df_11.show(10)

[(1, 'Manjot', 88, 60), (2, 'Saurav', 92, 73), (3, 'Sarv', 90, 81)]
+-----------+------+-----+-------+
|Roll_Number|  Name|Maths|Physics|
+-----------+------+-----+-------+
|          1|Manjot|   88|     60|
|          2|Saurav|   92|     73|
|          3|  Sarv|   90|     81|
+-----------+------+-----+-------+



In [15]:
student_df_11.agg({"Maths": 'avg'}).show()
student_df_11.agg({"Physics": 'avg'}).show()

+----------+
|avg(Maths)|
+----------+
|      90.0|
+----------+

+-----------------+
|     avg(Physics)|
+-----------------+
|71.33333333333333|
+-----------------+



In [17]:
from pyspark.sql.functions import *
student_df_11.select(avg("Maths")).collect()

[Row(avg(Maths)=90.0)]

In [18]:
from pyspark.sql.functions import when
student_df_11 = student_df_11.withColumn("Maths", when(student_df_11.Name == "Sarv", 99).otherwise(student_df_11.Maths))
student_df_11.show()

+-----------+------+-----+-------+
|Roll_Number|  Name|Maths|Physics|
+-----------+------+-----+-------+
|          1|Manjot|   88|     60|
|          2|Saurav|   92|     73|
|          3|  Sarv|   99|     81|
+-----------+------+-----+-------+



In [19]:
newRow = spark.createDataFrame([(4, "Mitansh", 90, 0)], schema = student_schema)
student_df_11 = student_df_11.union(newRow)
student_df_11.show()

+-----------+-------+-----+-------+
|Roll_Number|   Name|Maths|Physics|
+-----------+-------+-----+-------+
|          1| Manjot|   88|     60|
|          2| Saurav|   92|     73|
|          3|   Sarv|   99|     81|
|          4|Mitansh|   90|      0|
+-----------+-------+-----+-------+

